In [1]:
import requests

import urllib.request
import json 

In [2]:
data = ( 
    "How to summarize the employees by net revenue",
    "My data is look like below (usually more than",
    "<sql><sql-server>"
)

In [3]:
def send_json(x):
    Title, Body, Tags = x
    body = {
        'Title': Title, 
        'Body': Body,
        'Tags': Tags
        }
    myurl = 'http://127.0.0.1:8082/' + '/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

In [4]:
response = send_json(data)
print('предсказание', response)

предсказание 0.7913051672097864


Сделаем обработку массы запросов:

In [5]:
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score
from urllib import request, parse

In [6]:
df = pd.read_csv("./valid.csv")
df.head(10)

,Id,Title,Body,Tags,CreationDate,Y
0,34552974,How to get all the child records from differen...,I am having 4 different tables like \r\nselect...,<sql><sql-server>,2016-01-01 01:44:52,LQ_EDIT
1,34554721,Retrieve all except some data of the another t...,I have two table m_master and tbl_appointment\...,<php><mysql><sql><codeigniter><mysqli>,2016-01-01 08:43:50,LQ_EDIT
2,34555135,Pandas: read_html,<p>I'm trying to extract US states from wiki U...,<python><pandas>,2016-01-01 09:55:22,HQ
3,34555448,Reader Always gimme NULL,"I'm so new to C#, I wanna make an application ...",<sql-server><c#-4.0>,2016-01-01 10:43:45,LQ_EDIT
4,34555752,php rearrange array elements based on condition,basically i have this array:\r\n\r\n array(...,<php>,2016-01-01 11:34:09,LQ_EDIT
5,34557209,How do I make a constructor for a derived class?,<p>I am trying to make a constructor for a der...,<c++><inheritance><constructor><derived-class>,2016-01-01 14:58:52,LQ_CLOSE
6,34557363,how can i create a dynamic tow dimensional arr...,I am using c++ in my lesson and for solving a...,<c++>,2016-01-01 15:21:17,LQ_EDIT
7,34557587,Re-exporting ES6 modules in TS 1.7?,<p>I'm getting a bit lost in TS re-exports. Sa...,<typescript><ecmascript-6><es6-module-loader><...,2016-01-01 15:50:30,HQ
8,34558264,Fetch API with Cookie,<p>I am trying out the new Fetch API but is ha...,<cookies><fetch-api>,2016-01-01 17:21:19,HQ
9,34559136,Print list content in a given order,"<pre><code>lt = ['7,777.7', '777.7', '77,777.7...",<python><list><python-2.7><for-loop>,2016-01-01 19:08:12,LQ_CLOSE


In [7]:
df.fillna('', inplace=True)

In [8]:
df['Y'] = df['Y'].apply(lambda x: 0 if x == 'HQ' else 1)

In [9]:
df['Y'].value_counts()

1    10000
0     5000
Name: Y, dtype: int64

In [10]:
response = send_json(df[['Title', 'Body','Tags']].iloc[0,:])
print('предсказание', response)

предсказание 0.7913051672097864


In [11]:
N = 50

In [12]:
%%time
predictions = df[['Title', 'Body','Tags']
                ].iloc[:N].apply(lambda x: send_json(x), axis=1)

CPU times: total: 0 ns
Wall time: 461 ms


In [13]:
predictions.values[:5]

array([0.79130517, 0.79130517, 0.79130517, 0.79130517, 0.79130517])

In [14]:
y_test = df['Y']

In [15]:
import numpy as np

precision, recall, thresholds = precision_recall_curve(y_test[:N], predictions)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.7913051672097864, F-Score=0.750, Precision=0.600, Recall=1.000


In [16]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

0.5